# Renamed Binary
Detects the execution of a renamed binary often used by attackers or malware leveraging new Sysmon OriginalFileName datapoint.

## Rule Content
```
- title: Renamed Binary
  id: 36480ae1-a1cb-4eaa-a0d6-29801d7e9142
  status: experimental
  description: Detects the execution of a renamed binary often used by attackers or
    malware leveraging new Sysmon OriginalFileName datapoint.
  author: Matthew Green - @mgreen27
  date: 2019/06/15
  references:
  - https://attack.mitre.org/techniques/T1036/
  - https://mgreen27.github.io/posts/2019/05/12/BinaryRename.html
  - https://mgreen27.github.io/posts/2019/05/29/BinaryRename2.html
  tags:
  - attack.t1036
  - attack.defense_evasion
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      OriginalFileName:
      - cmd.exe
      - powershell.exe
      - powershell_ise.exe
      - psexec.exe
      - psexec.c
      - cscript.exe
      - wscript.exe
      - mshta.exe
      - regsvr32.exe
      - wmic.exe
      - certutil.exe
      - rundll32.exe
      - cmstp.exe
      - msiexec.exe
      - 7z.exe
      - winrar.exe
    filter:
      Image:
      - '*\cmd.exe'
      - '*\powershell.exe'
      - '*\powershell_ise.exe'
      - '*\psexec.exe'
      - '*\psexec64.exe'
      - '*\cscript.exe'
      - '*\wscript.exe'
      - '*\mshta.exe'
      - '*\regsvr32.exe'
      - '*\wmic.exe'
      - '*\certutil.exe'
      - '*\rundll32.exe'
      - '*\cmstp.exe'
      - '*\msiexec.exe'
      - '*\7z.exe'
      - '*\winrar.exe'
    condition: selection and not filter
  falsepositives:
  - Custom applications use renamed binaries adding slight change to binary name.
    Typically this is easy to spot and add to whitelist
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(file_name_original:("cmd.exe" OR "powershell.exe" OR "powershell_ise.exe" OR "psexec.exe" OR "psexec.c" OR "cscript.exe" OR "wscript.exe" OR "mshta.exe" OR "regsvr32.exe" OR "wmic.exe" OR "certutil.exe" OR "rundll32.exe" OR "cmstp.exe" OR "msiexec.exe" OR "7z.exe" OR "winrar.exe") AND (NOT (process_path.keyword:(*\\cmd.exe OR *\\powershell.exe OR *\\powershell_ise.exe OR *\\psexec.exe OR *\\psexec64.exe OR *\\cscript.exe OR *\\wscript.exe OR *\\mshta.exe OR *\\regsvr32.exe OR *\\wmic.exe OR *\\certutil.exe OR *\\rundll32.exe OR *\\cmstp.exe OR *\\msiexec.exe OR *\\7z.exe OR *\\winrar.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()